Today I need to finish the inplace operator PR. Yesterday I experimented with the `register_binary_operator_kernel`  function code by passing the third `output` argument as same as the first argument and it worked, yeah so for the first implementation I just need to call it like that.

_(The inplace operator call makes a lot of assignments and recasting which it doesn't need to do)_

```
-----------------------------IR DUMP: inplace_add_5-----------------------------
label 0:
    x = arg(0, name=x)                       ['x']
    $const0.2 = const(int, 5)                ['$const0.2']
    $0.3 = inplace_binop(rhs=$const0.2, lhs=x, fn=+) ['$0.3', '$const0.2', 'x']
    del x                                    []
    del $const0.2                            []
    x.1 = $0.3                               ['$0.3', 'x.1']
    del x.1                                  []
    del $0.3                                 []
    $const0.4 = const(NoneType, None)        ['$const0.4']
    $0.5 = cast(value=$const0.4)             ['$0.5', '$const0.4']
    del $const0.4                            []
    return $0.5                              ['$0.5']
```

* * *

The functions `register_binary_operator_kernel` and other such functions creates a registry entry to map operators with 


Okay ... the expression of the inplace operator is `inplace_binop(rhs=$const0.2, fn=+, lhs=x)` so the function it really takes in the this `+` operator, I tried some crude way to map it `inplace_kernel` but it is kind of not working out properly, first because the signature of both the `+` and `+=` is turing out to be the same

* * *
Okay in the afternoon I was successful in getting my code work, in the middle I was getting a strange error on futher investigation I noticed that code had
```python
args = args.append(args[0])
```
The problem here is that `append` returns `None` -_-.

Now, the way numba works is that it creates a mapping of `signature` + `function` to a kernel operation. In the python to IR conversion step it converts `+=` to `inplace_binop(rhs=$const0.2, fn=+, lhs=x)` and here the operator is `inplace_binop` but the function is `+`, so the it basically get maped to the same kernel operator as that of binary `+`. To circumvent this problem I created a diffent maps for `+=` like things and then I when I detected `inplace_binonp` I modified `+` to `+=`. In the inplace operation I modified code to make the output variable same as the first input variable. But mutating the IR is rather crude and the reviewer asked to find some other method instead.